In [ ]:
v_ll_batchcsvout="s3://wi-cred-datalake-dev-raw/transformed/scoring/outbound/batch/ll/2022/06/09/09/batch4_scoringdata20220609093904_3051938216.csv.out"
v_xg_batchcsvout="s3://wi-cred-datalake-dev-raw/transformed/scoring/outbound/batch/ll/2022/06/09/09/batch4_scoringdata20220609093904_3051938216.csv.out"
v_ll_realtimejsonl="s3://wi-cred-datalake-dev-raw/transformed/scoring/processing/realtime/ll/wi-mlops-lease-pric-ml-train-piln-lr-endpoint/AllTraffic/2022/06/08/11/09-14-802-13c75e13-7096-4935-b089-06e9f7b5113d.jsonl"
v_xg_realtimejsonl="s3://wi-cred-datalake-dev-raw/transformed/scoring/processing/realtime/ll/wi-mlops-lease-pric-ml-train-piln-lr-endpoint/AllTraffic/2022/06/08/11/09-14-802-13c75e13-7096-4935-b089-06e9f7b5113d.jsonl"

In [ ]:
v_ll_batchcsvout="s3://wi-cred-datalake-dev-raw/transformed/titanic/scoring/outbound/batch/ll/2022/08/17/03/scorewolabel20220817034014_130312380.csv.out"
v_xg_batchcsvout="s3://wi-cred-datalake-dev-raw/transformed/titanic/scoring/outbound/batch/xg/2022/08/17/03/scorewolabel20220817034014_3788033936.csv.out"
v_ll_realtimejsonl="s3://wi-cred-datalake-dev-raw/transformed/titanic/scoring/processing/realtime/ll/wi-mlops-titanic-ml-train-piln-lr-endpoint/AllTraffic/2022/08/17/14/06-35-563-c0ccc365-55f6-4a95-810f-40e9f30e3433.jsonl"
v_xg_realtimejsonl="s3://wi-cred-datalake-dev-raw/transformed/titanic/scoring/processing/realtime/xg/wi-mlops-titanic-ml-train-piln-xgboost-endpoint/AllTraffic/2022/08/17/14/06-34-184-5a829d89-9bc6-464a-b194-69fc9416df8e.jsonl"

In [ ]:
v_nlp_batchcsvout="s3://wi-cred-datalake-dev-raw/data/repscoreinput/train20221006144612.csv"

In [ ]:
import pandas as pd 
import random
def batchcsvtogtcsv(inpfile,opfilename):
    df = pd.read_csv(inpfile, header=None)
    df_gt = df.iloc[:,[3,17]]
    df_gt = df_gt.rename({0: 'InfID', 1: 'GroundTruthValue'}, axis=1)
    for i in range(len(df_gt)):
        df_gt.iloc[i,1] = random.randint(11500,22000)
    print(df_gt)
    df_gt.to_csv(opfilename+'.csv',index = False)  

In [ ]:
#GT NLP Turi
import pandas as pd 
import random
def batchcsvtogtcsv(inpfile,opfilename):
    df = pd.read_csv(inpfile, header=None)
    df_gt = df.iloc[:,[0,1]]
    df_gt = df_gt.rename({0: 'InfID', 1: 'GroundTruthValue'}, axis=1)
#     for i in range(len(df_gt)):
#         df_gt.iloc[i,1] = random.randint(11500,22000)
    print(df_gt)
    df_gt.to_csv(opfilename+'.csv',index = False)

In [ ]:
batchcsvtogtcsv(v_nlp_batchcsvout,"batchnlpgt")


In [13]:
#transfer file to respective location of Ground Truth
!aws s3 cp batchnlpgt.csv s3://wi-cred-datalake-dev-raw/transformed/nlp/monitoring/inbound/inpgroundtruth/batch/turi/

upload: ./batchnlpgt.csv to s3://wi-cred-datalake-dev-raw/transformed/nlp/monitoring/inbound/inpgroundtruth/batch/turi/batchnlpgt.csv


In [5]:
import pandas as pd
df=pd.read_csv("s3://wi-cred-datalake-dev-raw/data/repscoreinput/train20221006144612.csv")
df['text']=df['text'].str.replace(',','')
df.to_csv("textnwithoutcomma.csv")

In [7]:
!aws s3 cp textnwithoutcomma.csv s3://wi-cred-datalake-dev-raw/nlp/data/scoreinput/nlptextnwithoutcomma.csv

upload: ./textnwithoutcomma.csv to s3://wi-cred-datalake-dev-raw/nlp/data/scoreinput/nlptextnwithoutcomma.csv


In [ ]:
!pip install jsonlines

In [ ]:
# real time gt generation
import pandas as pd
import random
import jsonlines
import boto3
def realtimejsonltogtcsv(inpfile,outfilename):
    df = pd.DataFrame(columns = ['InfID', 'GroundTruthValue'])
    count = 0
    s3rsc=boto3.resource('s3')
    str1=inpfile.split('//',1)[-1]
    print(str1)
    bucket=str1.split('/',1)[0]
    key=str1.split('/',1)[1]
    localfilename="inpjsonfile.jsnol"
    s3rsc.Bucket(bucket).download_file(key,localfilename)
    with jsonlines.open(localfilename) as f:
        for line in f.iter():
            df.at[count,'InfID'] = line['eventMetadata']['inferenceId']
            df.at[count,'GroundTruthValue'] =  random.randint(11500,24000)
            count = count+1
           
    df.to_csv(outfilename+'.csv',index = False) 

In [ ]:
realtimejsonltogtcsv(v_ll_realtimejsonl,"realtimellgt")
realtimejsonltogtcsv(v_xg_realtimejsonl,"realtimexggt")
#same way run for xg batch

In [ ]:
!aws s3 cp batchllgt.csv s3://wi-cred-datalake-dev-raw/transformed/titanic/monitoring/inbound/inpgroundtruth/batch/ll/
!aws s3 cp batchxggt.csv s3://wi-cred-datalake-dev-raw/transformed/titanic/monitoring/inbound/inpgroundtruth/batch/xg/
!aws s3 cp realtimellgt.csv s3://wi-cred-datalake-dev-raw/transformed/titanic/monitoring/inbound/inpgroundtruth/realtime/ll/
!aws s3 cp realtimexggt.csv s3://wi-cred-datalake-dev-raw/transformed/titanic/monitoring/inbound/inpgroundtruth/realtime/xg/    

In [ ]:
!aws s3 cp s3://wi-cred-datalake-dev-raw/transformed/monitoring/reporting/datadrift/ s3://wi-cred-datalake-dev-raw/transformed/monitoring/reporting/drift/datadrift/ --recursive

In [ ]:
!aws s3 cp s3://wi-cred-datalake-dev-raw/transformed/monitoring/reporting/modeldrift/ s3://wi-cred-datalake-dev-raw/transformed/monitoring/reporting/drift/modeldrift/ --recursive